In [1]:
class PIDController():

    AUTO_MODE = 1
    OFF_MODE = 0

    def __init__(self, pgain=1, itime=1, dtime=0, auto_max=100,
                 auto_min=0, do_ifactor=False, anti_windup_backcalc=True,
                 beta=1, linearity=1):

        self.auto_min = auto_min
        self.auto_max = auto_max
        self.pgain = pgain
        self.itime = itime
        if self.itime:
            self.oneoveritime = 1 / self.itime
        else:
            self.oneoveritime = 0
        self.dtime = dtime
        self.dif = do_ifactor
        self.awb = anti_windup_backcalc

        self.accumulated_error = 0
        self.bump = 0
        self.last_output = 0
        self.last_error = 0
        self.last_pv = 0
        self.b = beta
        self.l = linearity

    def off_to_auto(self, pv, sp):
        """
        Calculate bump for off-to-auto transfer
        :param pv: current process temp to use for bumpless xfer calculation
        """
        self.man_to_auto(pv, sp, 0)

    def man_to_auto(self, pv, sp, op):
        err_for_pgain = (self.b*sp-pv)*(self.l+(1-self.l)*(abs(self.b*sp-pv))/100)
        uk0 = self.pgain * err_for_pgain
        self.bump = op - uk0
        self.last_pv = pv
        
    set_bump = man_to_auto

    def reset(self):
        self.accumulated_error = 0
        self.last_pv = 0
        self.last_error = 0        
    
    def step(self, pv, sp, dt=1):
        err = sp - pv
        dpv = pv - self.last_pv
        ierr = (err + self.last_error) / 2 * dt
        
        # SP_Range is taken to be 100 for both of the below lines
        # since range is calculated as +/- middle (?)
        # so -100-100 is actually range of 100 according to LV (??)
        err_for_pgain = (self.b*sp-pv)*(self.l+(1-self.l)*(abs(self.b*sp-pv))/100)
        if self.dif:
            ierr *= 1 / (1 + err * err * 10 / (100*100))  # Labview's ifactor thingy
        
        # bump (aka controller bias) isn't normally
        # included in Up, but no one else reads my 
        # code :)
        Up = self.bump + self.pgain * err_for_pgain
        Ui = self.oneoveritime * (ierr + self.accumulated_error) * self.pgain
        Ud = self.dtime * dpv * self.pgain
        Uk = Up + Ui + Ud
        
        # XXX debugging
        self.Ui = Ui * self.pgain
        self.Uk = Uk
        self.Ud = Ud * self.pgain
        self.Up = Up
        
        # Coercion & back calculation      
        back_calc = False
        if Uk > self.auto_max:
            Uk = self.auto_max
            back_calc = True
            am = self.auto_max
        elif Uk < self.auto_min:
            Uk = self.auto_min
            back_calc = True
            am = self.auto_min
            
        if back_calc and self.awb:
            ierr = self.itime / self.pgain * (am - Up - Ud) - self.accumulated_error
            
        self.accumulated_error += ierr
        self.last_output = Uk
        self.last_pv = pv
        self.last_error = err
        self.last_ierr = ierr
        
        return Uk

    def __repr__(self):
        return "Output: %.2f Pgain: %.1f Itime: %.2f AccumError: %.4f" % (self.last_output,
                                                                          self.pgain,
                                                                          self.itime,
                                                                          self.accumulated_error)
    __str__ = __repr__
    

In [2]:
def assert_equal(a,b):
    assert a == b, (a, b)
def test_trap_integration1():
    data = [
        (0, 1, .5),
        (0, 2, 2),
        (0, 3, 4.5)
    ]
    p = PIDController(1,1)
    for pv, sp, op in data:
        p.off_to_auto(pv, sp)
        res_op = p.step(pv, sp)
        assert_equal(p.last_error, sp)
        assert_equal(p.accumulated_error, op)
        assert_equal(res_op, op)
test_trap_integration1()

def check_Ui_backcalc(it, pg, Up, Ud, ierr, accum, val):
    Ui2 = it * (ierr + accum) * pg
    Uk2 = Up + Ud + Ui
    Uk2 = round(Uk2, 8)
    am = round(val, 8)
    assert Uk2 == am, (Up, Ud, Ui, Uk2, val)

In [3]:
def m2s(m):
    return m*60
def s2m(s):
    return s/60
def h2s(h):
    return m2s(h*60)

In [4]:
from types import FunctionType, MethodType
def printdir(o):
    for a in dir(o):
        if not a.startswith("__"):
            v = getattr(o, a)
            if not isinstance(v, (FunctionType, MethodType)):
                print(a, v)

In [5]:
from collections import deque
class DelayBuffer(deque):
    def __init__(self, delay=30, startvalue=0):
        delay = int(delay)
        self.delay = delay
        super().__init__(startvalue for _ in range(delay + 1))

    def cycle(self, hd):
        self[0] = hd
        self.rotate(1)
        return self[0]

In [125]:
import numpy as np
class DOProcess():
    def __init__(self, delay, initial=20, env=18.5, g=0.0019254, k=-0.001579031):
        """
        :param g: gain in units of C/min/%
        :param k: decay rate in units of C/min/dT
        """
        self.tdelay = DelayBuffer(delay, initial).cycle
        self.g = g / 60
        self.k = k / 60
        self.env = env
        
    def step(self, pv, op):
        op = self.tdelay(op)
        dT = pv - self.env
        decay = self.k * dT
        gain = self.g * op
        dpv = decay + gain
        return pv + dpv
    
class Delay():
    def __init__(self, delay, initial, pc_at_delay=0.95):
        assert 0 < pc_at_delay <= 1, "don't be a retard"
        if delay:
            self.df = np.log(1/(1-pc_at_delay)) / delay
        else:
            self.df = 1
        self.sink = initial / self.df  - initial # fill the sink
        if self.sink < 0:
            self.sink = 0
        
    def delay(self, op, dt):
        self.sink += op * dt
        dNdt = self.df * self.sink * dt
        self.sink -= dNdt
        return dNdt
        
    
class DOProcess2():
    def __init__(self, delay, initial=20, g=0.0019254, k=-0.001579031):
        """
        :param g: gain in units of C/min/%
        :param k: decay rate in units of C/min/dT
        """
        self.heatsink = Delay(delay, initial, 0.95)
        self.g = g / 60
        self.k = k / 60
        
    def step(self, pv, op):
        op = self.heatsink.delay(op, 1)
        dT = 100 - pv
        decay = self.k * dT
        gain = self.g * op
        dpv = decay + gain
        return pv + dpv


class HeadspaceProcess():
    def __init__(self, vol, main_gas, co2R=0, n2R=0, o2R=0, co2A=0, n2A=0.78, o2A=0.21):
        """ 
        vol: volume headspace (L)
        main_gas: main gas flow rate (L/min)
        co2: CO2 request (%)
        n2: N2 request (%)
        
        Calculate headspace concentration. DT = 1 second. 
        """
        self.vol = vol
        self.main_gas = main_gas
        self.co2R = co2R
        self.n2R = n2R
        self.o2R = o2R
        self.airR = 1 - (co2R+n2R+o2R)
        self.co2A = co2A
        self.n2A = n2A
        self.o2A = o2A
        self.airA = 1 - (co2A+n2A+o2A)
    
    def calc_gas(self):
        """ Calculate new headspace gas concentration.
        Assumptions:
            Mixing time is 0 seconds, i.e. gas mixture is always perfectly mixed
            Gas lost from headspace is always equal to gas entering headspace
            Concentration of individual gasses lost match headspace concentration of gasses. 
            Mass transfer coefficient of Air <-> liquid k <<< Mg (no concentration change across liquid boundary)
            Air O2: 20.95%, N2: 78.09%, CO2=0.04%, other = 1-(o2+n2+co2)
        """
        mg = self.main_gas / 60
        
        # volume of gas lost
        co2L = mg * self.co2A
        n2L = mg * self.n2A
        o2L = mg * self.o2A
        airL = mg * self.airA
        
        # volume of gas gained
        # airG is non-corrected volume of air added
        co2G = mg * self.co2R
        n2G = mg * self.n2R
        o2G = mg * self.o2R
        airG = mg * self.airR
        
        # air contribution to gases
        # airG is corrected to reflect
        # the volume of non-tracked gases
        # ie the 1% of trace elements or so
        co2G += 0.0004 * airG
        o2G += 0.2095 * airG
        n2G += 0.7809 * airG
        airG = (1 - (0.0004+.2095+.7809)) * airG
        
        # new volume of each gas
        co2V = co2G - co2L + self.co2A * self.vol
        n2V = n2G - n2L + self.n2A * self.vol
        o2V = o2G - o2L + self.o2A * self.vol
        airV = airG - airL + self.airA * self.vol
        
        # finally, new percentages
        self.co2A = co2V / self.vol
        self.n2A = n2V / self.vol
        self.o2A = o2V / self.vol
        
        # sanity check
        assert self.vol == round(co2V + n2V + o2V+airV, 6), (self.vol, co2V, n2V, o2V, sum((co2V, n2V, o2V)))
        
        
    

In [126]:
h = HeadspaceProcess(30, 0.5, 0.07, .10, .40)
def step():
    h.calc_gas()

In [128]:
y1 = []

In [21]:
def do_sim(p, i, d, delay, amax, amin, end, op=0, pv=20, sp=37, 
             g=0.0019254, k=-0.001579031, mode='m2a', beta=1):
    pid = PIDController(p,m2s(i),m2s(d),amax,amin, do_ifactor=True, beta=beta)
    if mode == 'm2a':
        pid.man_to_auto(pv, sp, op)
    else:
        pid.man_to_auto(pv, pv, op)
    proc = DOProcess2(delay, op, g=g, k=k)
    t = 0
    data = [(t, pv, op)]
    while True:
        t += 1
        op = pid.step(pv, sp)
        pv = proc.step(pv, op)
        data.append((t, pv, op))
        if t >= end:
            break
    return data

    

In [22]:
def paste(cells, data, offset=0):
    with screen_lock(xl):
        cells.Range(cells(1,1+offset), cells(1, 3+offset)).Value = [("T", "PV", "OP")]
        topleft = cells(2,1+offset)
        bottomright = topleft.Offset(len(data), len(data[0]))
        cells.Range(topleft, bottomright).Clear()
        cells.Range(topleft, bottomright).Value = data
    
def clear(cells, data, offset=0):
    if not data: return
    with screen_lock(xl):
        topleft = cells(2,1+offset)
        bottomright = topleft.Offset(len(data), len(data[0]))
        cells.Range(topleft, bottomright).Clear()

In [23]:
from officelib.xllib import *
xl = Excel()

wb = xl.ActiveWorkbook
wb = wb or xl.Workbooks.Add()

ws = xl.ActiveSheet
ws = ws or wb.Worksheets.Add()

cells = ws.Cells
cell_range = ws.Cells.Range

In [24]:
offset = 0
p = 60
i = 35
data = []
delay=m2s(15)

In [43]:
clear(cells, data, 0)
data = do_sim(p=1, i=10, d=0, delay=m2s(5), k=0.0005,
                    amax=100, amin=0, end=m2s(480), op=0, 
                    pv=30, sp=40, mode='a2a', beta=0)
paste(cells, data)

In [38]:
50/(2*24*60*60)

0.00028935185185185184